![RookEye](https://raw.githubusercontent.com/AlienWolfX/thesis-chess/refs/heads/main/img/rookeye.ph.png)

Yolov8 Training notebook for our thesis


Github: [AlienWolfX](https://github.com/AlienWolfX)

          

## Installing YOLOv8

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.2.103 -q

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

## Downloading dataset from roboflow

In [ ]:
!mkdir -p rookeye
%cd rookeye

!pip install roboflow --quiet

import roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="i4Pfe2x0BiQamj9K5l9a") # Ayaw pag ikalat 🤫
project = rf.workspace("rookeye").project("rookeye")
version = project.version(21) # Gamit latest
dataset = version.download("yolov8")

## Training


In [ ]:
import yaml

with open(f"{dataset.location}/data.yaml", 'r') as f:
    dataset_yaml = yaml.safe_load(f)
dataset_yaml["train"] = "../train/images"
dataset_yaml["val"] = "../valid/images"
dataset_yaml["test"] = "../test/images"
with open(f"{dataset.location}/data.yaml", 'w') as f:
    yaml.dump(dataset_yaml, f)

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=50 plots=True batch=16

## Viewing Stats

In [ ]:
from IPython.display import Image, display

# Display the first image
display(Image(filename='runs/detect/train/confusion_matrix.png', width=600))
print("")

# Display the second image
display(Image(filename='runs/detect/train/results.png', width=600))
print("")

# Display the third image
display(Image(filename='runs/detect/train/val_batch0_pred.jpg', width=600))
print("")


## Validate Custom Model

In [ ]:
!yolo task=detect mode=val model=runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

## Inference with Custom Model (images)

In [ ]:
!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

**NOTE:** Let's take a look at few results.

In [ ]:
import glob
import os

from IPython.display import Image, display

base_path = 'runs/detect/'

subfolders = [os.path.join(base_path, d) for d in os.listdir(base_path)
              if os.path.isdir(os.path.join(base_path, d)) and d.startswith('predict')]

latest_folder = max(subfolders, key=os.path.getmtime)

image_paths = glob.glob(os.path.join(latest_folder, '*.jpg'))[:14]

for image_path in image_paths:
    display(Image(filename=image_path, width=600))
    print("\n")


## Inference with Custom Model (video)

In [ ]:
!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt conf=0.25 source=0 save=True

# Downloading Weights (Not applicable if run locally)

In [ ]:
from google.colab import files
files.download('runs/detect/train/weights/best.pt')